In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from datetime import datetime

from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

In [12]:
data = pd.read_csv('lemmatized_no_noise.csv').drop('Unnamed: 0', axis = 1)
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
3925,dying,2015-05-12,1419,16,False,False,забавно человек говорить значить очень многое ...
4781,dying,2015-04-18,1055,11,False,False,история злой инн инна добрый
5531,dying,2015-03-29,2164,16,False,False,история грустный лена лена весело


# пытаемся анализировать тональность текста постов:

Мы решили проанализировать тональность текста постов, чтобы посмотреть, будет ли количество лайков зависеть от позитивной, негативной или нейтральной тональности. Для этого мы решили воспользоваться библиотекой TextBlob, который определяет тональность текста. Признаку тональность мы присваивали 4 значения: "Положительная", "отрицательная", "нейтральная" а также "некорректный тип", если при обработке ячейки возникали проблемы с ее форматом. 

После этого у нас из 9809 значений получилось 10 значений положительной тональности, 3 отрицательной, 9772 нейтральной и 24 некорректного типа. Это значит, что тональность определяется плохо. 

Поэтому мы решили взять не лемматизированные тексты постов, а изначальные и посмотреть, возможно, показатели улучшаться. Действительно, так и получилось, но эти изменения были слишком незначительными и нам по-прежнему надо было улучшать качество определения тональности постов. 

Мы предположили, что библиотека TextBlob лучше будет работать на данных на английском, а не русском языке. Для этого мы с помощью Excel перевели наши данные с русского на английский язык.

Изначально мы попытались перевести текст в Питоне через встроенные библиотеки-переводчики (например, googletranslator). Однако это потребовало огромного количества времени (несколько часов) из-за почти 10 000 обращений к переводчику. Поэтому было принято решение пойти другим путем, а именно перевести тексты в Excel. Так как в оригинальном приложении отсутствовала функция перевода, пришлось искать дополнительное ПО. Решением оказалась надстройка !SEMTools, которая тоже обращалась к гугл переводчику, но быстрее, чем библиотека в Python (30-40 минут против нескольких часов). С помощью найденного инструмента мы сначала перевели данные в excel, там перевели текст через формулу, а потом обратно конвертнули файл в csv-файл, с которым и работали далее.

После этого мы снова попытались определить тональность поста уже на данных на английском языке, и наши показатели заметно улучшились. Теперь постов положительной тональности стало 3730, отрицательной 2036 и нейтральной 4019. Количество постов некорректного типа по-прежнему не поменялось. 

Далее мы решили посмотреть, сколько в среднем лайков набирают посты с каждой тональностью. Получилось, что больше всего лайков набирают посты с нейтральной тональностью, а меньше всего с отрицательной, однако статистически эти различия незначимы.

In [14]:
# Определение функции для определения тональности текста
def get_sentiment(text):
    if isinstance(text, str):  # Проверка на тип значения
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity
        if sentiment > 0:
            return 'Положительная'
        elif sentiment < 0:
            return 'Отрицательная'
        else:
            return 'Нейтральная'
    else:
        return 'Некорректный тип'


In [27]:
data['тональность'].value_counts()

Нейтральная         9772
Некорректный тип      24
Положительная         10
Отрицательная          3
Name: тональность, dtype: int64

In [38]:
data_0 = pd.read_csv('data.csv').drop('Unnamed: 0', axis = 1)

In [39]:
data_0['тональность'] = data_0['text'].apply(get_sentiment)
data_0

,author,date,text,n_likes,n_reposts,with_photo,is_repost?,тональность
0,dying,17 авг 2015,хочу видеть тебя каждый день,14351,325,False,False,Нейтральная
1,dying,17 авг 2015,"Живёшь живёшь, и тут в твоей жизни появляется ...",7098,144,False,False,Нейтральная
2,dying,17 авг 2015,- По кому скучаешь?\n- По Наташе,6439,121,False,False,Нейтральная
3,dying,17 авг 2015,"и дай Бог каждому быть с тем, с кем сердце не ...",8469,221,False,False,Нейтральная
4,dying,17 авг 2015,Моя Арина милашка\nМоя Арина бандитка\nМоя Ари...,3704,71,False,False,Нейтральная
...,...,...,...,...,...,...,...,...
9984,dying,30 окт 2014,"- Как тебя Земля носит?\n- Ну, гравитация и ты...",263,7,False,False,Нейтральная
9985,dying,30 окт 2014,сохрани нежность,67,0,False,False,Нейтральная
9986,dying,30 окт 2014,"Если у тебя на лице прыщики, не комплексуй. По...",133,4,False,False,Нейтральная
9987,dying,30 окт 2014,"- Вы такая милая пара! Когда я вас вижу, вы вс...",636,12,False,False,Нейтральная


In [40]:
data_0['тональность'].value_counts()

Нейтральная         9692
Некорректный тип     180
Положительная        111
Отрицательная          6
Name: тональность, dtype: int64

In [41]:
translated = pd.read_csv('translated posts.csv').drop('Unnamed: 0', axis = 1)
translated.head(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text,translation
0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,want to see every day
1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,live live your life appear polina
2,dying,2015-08-17,6439,121,False,False,скучать наташа,miss natasha


In [42]:
translated['тональность'] = translated['translation'].apply(get_sentiment)
translated.head(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text,translation,тональность
0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,want to see every day,Нейтральная
1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,live live your life appear polina,Положительная
2,dying,2015-08-17,6439,121,False,False,скучать наташа,miss natasha,Нейтральная


In [43]:
translated['тональность'].value_counts()

Нейтральная         4019
Положительная       3730
Отрицательная       2036
Некорректный тип      24
Name: тональность, dtype: int64

In [44]:
# Присвоение числовых значений типам тональности
mapping = {'Положительная': 1, 'Отрицательная': -1, 'Нейтральная': 0}
translated['tonality_numbers'] = translated['тональность'].map(mapping)

Посмотрим на средние значение по лайкам в зависимости от тональности текста.

In [45]:
translated['n_likes'].mean()

1106.163727189316

In [46]:
translated[translated.tonality_numbers == 1]['n_likes'].mean()

1106.3195710455764

In [47]:
translated[translated.tonality_numbers == 0]['n_likes'].mean()

1112.3471012689724

In [48]:
translated[translated.tonality_numbers == -1]['n_likes'].mean()

1095.6335952848724

Кажется, что нейтральные посты набирают больше всего лайков. Проверим это, используя тест Уэлча (опять же, мы не знаем равны ли дисперсии и предполагаем нормальность выборок)

$$
H_0: \mu_{neutral} = \mu_{not\_neutral}
$$
$$
H_1: \mu_{neutral} > \mu_{not\_neutral}
$$

In [49]:
neutral = translated[translated.tonality_numbers == 0].n_likes
not_neutral = translated[translated.tonality_numbers != 0].n_likes

In [50]:
stats.ttest_ind(neutral, not_neutral, equal_var= False, alternative= 'greater')

Ttest_indResult(statistic=0.6734077371478718, pvalue=0.25035388585559026)

Получили p-value примерно равное 0.25. Получается нет оснований отвергать нулевую гипотезу. Получается, нейтральные посты в среднем набирают столько же лайков, сколько и все остальные.

Дополнительно, мы решили проверить, набирают ли негативные посты меньше лайков, чем все остальные. 

$$
H_0: \mu_{negative} = \mu_{not\_negative}
$$
$$
H_1: \mu_{negative} < \mu_{not\_negative}
$$

In [53]:
negative = translated[translated.tonality_numbers == -1].n_likes
not_negative = translated[translated.tonality_numbers != -1].n_likes

In [54]:
stats.ttest_ind(negative, not_negative, equal_var=False, alternative='less')

Ttest_indResult(statistic=-0.74141439087811, pvalue=0.22924751848380254)

Получили p-значение равное 0.23. Нет оснований отвергать нулевую гипотезу о том, что негативные посты в среднем набирают столько же лайков, сколько и все остальные